In [ ]:
import pandas as pd
import json
from tqdm import tqdm
import ast
import numpy as np
from collections import deque
from pandas import DataFrame
from collections import defaultdict
import pdb

In [ ]:
def str2list(x):
    if len(x) > 0:
        if x[0] != '[':
            list = [x]
        else: 
            list = ast.literal_eval(x)
            if len(list) == 0: list = ['Empty']
    else: list = ['Empty']

    return list

In [ ]:
basic_info = pd.read_csv('/opt/ml/wine/data/basic_info_total.csv', encoding='utf-8-sig')
with open('/opt/ml/wine/code/feature_map/item2idx.json','r') as f:
    item2idx = json.load(f)

In [ ]:
item_data = pd.read_csv('/opt/ml/wine/server/data/item_df_allfeature.csv', encoding='utf-8-sig')
item_data = item_data[item_data['item_id'].isna()==False]
item_data.drop_duplicates(subset='item_id', inplace = True)

In [ ]:
wine_df = pd.read_csv('/opt/ml/wine/data/wine_df.csv', encoding='utf-8-sig')

In [ ]:
wine_df['grape'].fillna('[]', inplace = True)
wine_df['grape'] = wine_df['grape'].apply(lambda x: str2list(x))

In [ ]:
wine_df['item_id'] = wine_df['url'].map(item2idx)
wine_df = wine_df[wine_df['item_id'].isna()==False]
wine_df['item_id'] = wine_df['item_id'].astype(int).astype('category')

In [ ]:
basic_info['item_id'] = basic_info['url'].map(item2idx)
basic_info = basic_info[basic_info['item_id'].isna()==False]
basic_info['item_id'] = basic_info['item_id'].astype(int).astype('category')

In [ ]:
basic_info.set_index('item_id', inplace= True)
item_data.set_index('item_id', inplace= True)

In [ ]:
no_key = 0
for item_id in tqdm(item_data.index):
    try:
        item_data.loc[item_id,'winery'] = basic_info.loc[item_id,'winery']
        item_data.loc[item_id,'grape'] = basic_info.loc[item_id,'grapes']
        item_data.loc[item_id,'country'] = basic_info.loc[item_id,'country']
        item_data.loc[item_id,'region'] = basic_info.loc[item_id,'region1']
        item_data.loc[item_id,'region2'] = basic_info.loc[item_id,'region2']
        item_data.loc[item_id,'region3'] = basic_info.loc[item_id,'region3']
        item_data.loc[item_id,'region4'] = basic_info.loc[item_id,'region4']
        item_data.loc[item_id,'wine_style'] = basic_info.loc[item_id,'wine_style']
        item_data.loc[item_id,'allergens'] = basic_info.loc[item_id,'allergens']
        item_data.loc[item_id,'alcohol content'] = basic_info.loc[item_id,'alcohol content']
    except KeyError:
        try:
            item_data.loc[item_id,'grape'] = wine_df.loc[item_id,'grape']
        except:
            no_key +=1
print(no_key)

In [ ]:
item_data['item_id'] = item_data.index

In [ ]:
item_data = pd.read_csv('/opt/ml/wine/data/item_df_allfeature.csv',encoding='utf-8-sig')
item_data.rename(columns = {'region':'region1'}, inplace= True)

In [ ]:
with open('/opt/ml/wine/data/sample_vectors.json','r') as f: 
    sample_vec = json.load(f)

In [ ]:
import re

def keep_only_english(text):
    try:
        english_text = re.sub(r'[^a-zA-Z\s]', '', text)
        return english_text.lower().strip()
    except: return None

In [ ]:
def find_vectors(columns_name : deque, grouped_vectors: DataFrame):
    column, name = columns_name.popleft()
    if name is not None:
        grouped_vectors = grouped_vectors.query(f"{column} == '{name}'")
        if columns_name:
            vector = find_vectors(columns_name, grouped_vectors.drop(column,axis = 1))
        else: 
            vector = grouped_vectors.vectors.mean()
            
        return vector
    else:
        return grouped_vectors.vectors.mean()

def get_item_vector(df, vector_path):
    df = df.sort_values(by='item_id', ascending=True)
    with open(vector_path,'r') as f: 
        vectors = json.load(f)
    vector_list = []

    for id in tqdm(item_data.item_id):
        id = str(id)
        if id in vectors.keys():
            vector_list.append(np.array(vectors[id]))
        else:
            vector_list.append(None)
    df['vectors'] = vector_list
    return df


In [ ]:
def fill_vectors(df : DataFrame, vector_path: str):
    df.drop_duplicates(subset='item_id', keep='first', inplace=True)
    columns_to_check = df.columns.drop('item_id')
    df.dropna(subset=columns_to_check, how='all', inplace=True)
    df = get_item_vector(df, vector_path)

    for col in ['country','region1', 'winetype', 'wine_style','region']:
        if col in df.columns:
            df[col] = df[col].apply(keep_only_english)

    grouped_vectors = df[df.vectors.isna()==False].groupby([
        'country',
        'region1', 
        'winetype',
        'wine_style'
    ]).agg({'vectors': 'mean'}).reset_index()

    non_vectors = df[df['vectors'].isna()==True]
    non_vectors_cols = list(grouped_vectors.columns)
    non_vectors_cols.append('item_id')
    non_vectors = non_vectors.loc[:, non_vectors_cols]
    vectors = []

    for index, row in tqdm(non_vectors.iterrows()):
        columns_name = deque()

        for column, name in zip(row.keys(), row.values):
            if column != 'vectors':
                columns_name.append((column, name))
            else: break
        vector = find_vectors(columns_name, grouped_vectors)
        vectors.append(vector)
    
    non_vectors['vectors'] = vectors

    not_filled = non_vectors[non_vectors['vectors'].isna()]
    filled = non_vectors[non_vectors['vectors'].notna()]

    mean_vector = filled['vectors'].mean()

    not_filled['vectors'] = [mean_vector for _ in range(len(not_filled))]
    
    filled_total = pd.concat([filled, not_filled], axis=0)

    no_vectors = df[df['vectors'].isna()]
    no_vectors.drop('vectors', axis=1, inplace=True)
    no_vectors.reset_index(drop=True, inplace=True)  # Use drop=True to reset the index without keeping the old index
    no_vectors = no_vectors.sort_values(by='item_id', ascending=True)

    yes_vectors = df[df['vectors'].notna()]
    yes_vectors.reset_index(drop=True, inplace=True)

    if len(set(no_vectors['item_id']).intersection(set(yes_vectors['item_id']))) != 0:
        shared_item_ids = set(no_vectors['item_id']).intersection(set(yes_vectors['item_id']))
        yes_vectors = yes_vectors[~yes_vectors['item_id'].isin(shared_item_ids)]
        
    filled_total.reset_index(drop = True, inplace = True)
    filled_total = filled_total.sort_values(by='item_id', ascending=True)

    no_vectors.set_index('item_id', inplace = True)
    no_vectors['item_id'] = no_vectors.index
    
    yes_vectors.set_index('item_id', inplace = True)
    yes_vectors['item_id'] = yes_vectors.index

    filled_total.set_index('item_id', inplace = True)
    filled_total['item_id'] = filled_total.index
    
    no_vectors['vectors'] = filled_total['vectors']
    no_vectors.reset_index(drop = True, inplace = True)
    yes_vectors.reset_index(drop = True, inplace = True)

    item_data_with_vectors = pd.concat([no_vectors,yes_vectors], axis=0)
    print('item_data_with_vectors')
    print(len(item_data_with_vectors.drop_duplicates(subset='item_id')), len(item_data_with_vectors))
    
    item_data_with_vectors = item_data_with_vectors.sort_values(by='item_id', ascending=True).reset_index(drop = True)
    return item_data_with_vectors




In [ ]:
item_data_with_vectors = fill_vectors(item_data, '/opt/ml/wine/data/wine_vector.json')

In [ ]:
item_data_with_vectors.to_csv('/opt/ml/wine/data/item_data_vec.csv', index = False, encoding = 'utf-8-sig')

In [ ]:
item_data_with_vectors = pd.read_csv('/opt/ml/wine/data/item_data_sample_vec.csv', encoding = 'utf-8-sig')

In [5]:
def string2array(x):
    x = x.replace('\n', '').strip('[]')
    x_list = [float(i) for i in x.split(' ') if len(i) != 0]
    return np.array(x_list)

In [ ]:
item_data_with_vectors['vectors'] = item_data_with_vectors['vectors'].apply(string2array)

In [ ]:
wine_vectors = []
for vector in item_data_with_vectors['vectors']:
    wine_vectors.append(vector)
wine_vectors = np.array(wine_vectors)

In [ ]:
item_to_fill = item_data_with_vectors.iloc[83,:]


In [ ]:
def count_grape(data : pd.DataFrame):
    dict = defaultdict(float)
    for grapes, dist in zip(data.grape, data.distance):
        try: grapes = ast.literal_eval(grapes)
        except: pass

        for grape in grapes:
            try: dict[grape.lower()] += dist
            except TypeError as t: continue
    return [max(dict , key=lambda k: dict[k])]

def count_pairing(data : pd.DataFrame):
    dict = defaultdict(float)
    for pairings, dist in zip(data.pairing, data.distance):
        for menu in pairings.split(' '):
            try: dict[menu] += dist
            except TypeError as t: continue
    return max(dict , key=lambda k: dict[k])

def count_most_str(data : pd.DataFrame, column):
    dict = defaultdict(float)
    for feat, dist in zip(data[column], data.distance):
        try:
            dict[feat] += dist
        except TypeError as t:
            continue
    return max(dict , key=lambda k: dict[k])

def count_most_cont(data : pd.DataFrame, column):
    dict = defaultdict(float)
    for feat, dist in zip(data[column], data.distance):
        try:
            dict[feat] += dist
        except TypeError as t:
            continue
    return max(dict , key=lambda k: dict[k])

In [ ]:
from collections import defaultdict
def most_close(sim_items : DataFrame):
    result = {}
    sim_items.dropna(inplace=True)
    for col in sim_items.columns:
        if col == 'pairing':
            result[col] = count_pairing(sim_items)
        elif col == 'grape':
            result[col] = count_grape(sim_items)
        elif col in ['price', 'wine_rating', 'num_votes',
                     'Red Fruit', 'Tropical', 'Tree Fruit', 'Oaky', 'Ageing', 'Black Fruit',
                     'Citrus', 'Dried Fruit', 'Earthy', 'Floral', 'Microbio', 'Spices',
                     'Vegetal', 'Light', 'Bold', 'Smooth', 'Tannic', 'Dry', 'Sweet', 'Soft',
                     'Acidic', 'Fizzy', 'Gentle']:
            result[col] = count_most_cont(sim_items, col)
        else:
            result[col] = count_most_str(sim_items, col)
    return result


In [ ]:
import faiss
import pdb
from pandas import Series

wine_vectors = []
for vector in item_data_with_vectors['vectors']:
    wine_vectors.append(vector)
wine_vectors = np.array(wine_vectors)


def find_most_sim_item(df : DataFrame, to_fill_item_id: int, wine_vectors : np.array):

    ###index should be wine_id/item_id
    item_to_fill = item_data_with_vectors.loc[to_fill_item_id,:]

    item_vector = item_to_fill.vectors
    try:
        None_col = list(item_to_fill.index[item_to_fill.isna()])
    except: pdb.set_trace()
    None_col.append('distance')

    item_data_with_vectors.set_index('item_id', inplace=True)
    item_data_with_vectors['item_id'] = item_data_with_vectors.index
    item_ids = list(item_data_with_vectors['item_id'])
    vector_dimension = item_vector.shape[0]

    index = faiss.IndexFlatL2(vector_dimension)
    index = faiss.IndexIDMap2(index)
    index.add_with_ids(wine_vectors, item_ids)

    # Faiss expects the query vectors to be normalized
    to_search = np.expand_dims(item_vector, axis=0)
    to_search = np.ascontiguousarray(to_search.astype(np.float32))
    faiss.normalize_L2(to_search)

    k = index.ntotal
    distances, searched_wine_ids = index.search(to_search, k=20)

    result = []
    for ids, dists in zip(searched_wine_ids[0], distances[0]): 
        result.append((ids, dists))

    sim_items = item_data_with_vectors.loc[[x[0] for x in result], :]
    sim_items['distance'] = 0
    sim_items = sim_items.loc[:, None_col]
    
    
    for id, dist in result: sim_items.loc[id, 'distance'] = 1/dist

    to_fill = most_close(sim_items)
    
    for col, val in to_fill.items():
        item_data_with_vectors.loc[to_fill_item_id, col] = val

    return item_data_with_vectors
    

In [ ]:
for id in tqdm(item_data_with_vectors.index):
    item_data_with_vectors = find_most_sim_item(item_data_with_vectors, id, wine_vectors)

In [ ]:
def count_most_cont(data : pd.DataFrame, column):
    dict = defaultdict(float)
    data.dropna(inplace = True)
    total_sum = data['distance'].sum()

    # Make the smallest value to 1 and scale other values accordingly
    data['distance'] = data['distance'] / total_sum

    result = 0
    for feat, dist in zip(data[column], data.distance):
        try:
            result += dist * feat
        except TypeError as t:
            continue
    return result

In [ ]:
item_data = pd.read_csv('/opt/ml/wine/data/item_data_vec.csv', encoding='utf-8-sig')

In [6]:
item_data['vectors'] = item_data['vectors'].apply(string2array)

In [17]:
def bayesian_average(df, min_votes=20, prior_rating=None):
    v = df['num_votes']
    R = df['wine_rating']
    
    if prior_rating is None:
        C = df['wine_rating'].mean()  # Mean rating as the prior rating
    else:
        C = prior_rating

    m = min_votes

    df['popularity_adjusted_rating'] = ((v / (v + m)) * R) + ((m / (v + m)) * C)
    return df

In [28]:
min_votes = 20
item_data = bayesian_average(item_data, min_votes)
# Calculate Bayesian average and add it as a new column in the DataFrame

In [29]:
item_data['popularity_adjusted_rating']

0        3.681067
1        4.147625
2        4.006416
3        3.619521
4        3.802277
           ...   
74917    3.506071
74918    4.082210
74919    4.220119
74920    4.306937
74921    4.493291
Name: popularity_adjusted_rating, Length: 74922, dtype: float64

In [31]:
item_data.sort_values(by = 'popularity_adjusted_rating', ascending= False).head()

,country,region1,winery,winetype,grape,name,vintage,house,price,wine_rating,...,Acidic,Fizzy,Gentle,region2,region3,region4,item_id,vectors,bayesian_avg,popularity_adjusted_rating
22819,unitedstates,california,abreu,dessertwine,['Pinot Noir'],Sauternes 1997,1997.0,Château_d'Yquem,256.99,4.8,...,0.00,0.00,0.00,North_Coast,Napa_County,Napa_Valley,22819,"[0.0913422071, 0.507825071, 0.142717776, 0.144...",4.8,4.792745
16386,france,bordeaux,château d'yquem,dessertwine,['Sauvignon Blanc'],Sauternes 1990,1990.0,Château_d'Yquem,256.99,4.8,...,0.00,0.00,0.00,Sauternes,NaN,NaN,16386,"[0.0913422071, 0.507825071, 0.142717776, 0.144...",4.8,4.792163
51528,france,bourgogne,NaN,redwine,['Malbec'],Unico 1999,1999.0,Vega_Sicilia,782.00,4.8,...,77.50,0.00,0.00,NaN,NaN,NaN,51528,"[0.10880302, 0.48559926, 0.11923454, 0.0442084...",4.8,4.791409
25348,italy,northernitaly,ferrari,redwine,['Cabernet Sauvignon'],Unico 2010,2010.0,Vega_Sicilia,678.00,4.8,...,46.56,55.57,44.43,Trentino-Alto_Adige,Trento,NaN,25348,"[0.10061615, 0.557006955, 0.141965389, 0.13747...",4.8,4.790529
16855,italy,centralitaly,nevio,redwine,['Pinot Noir'],Unico 1996,1996.0,Vega_Sicilia,478.95,4.8,...,0.00,0.00,0.00,Abruzzo,Montepulciano_d'Abruzzo,NaN,16855,"[0.105900352, 0.487247233, 0.140899844, 0.1019...",4.8,4.788932
